# Function Calling

This Notebook Covers the steps needed to implement Function calling capabilities with Sambastudio and Sambaverse models, leveraging Langchain Tools and Langchain integrations 

We also provide the [FunctionCallingLlm module](../src/function_calling.py) and the [usage notebook](./usage.ipynb), these can be used as a quick start to use function calling, and this notebook can be used as a guide for further customizing your function calling model and tools 

In [377]:
import os
import re
import sys
import json
import operator
from pprint import pprint
from datetime import datetime
from dotenv import load_dotenv
from typing import Optional, Union
from langchain_community.llms.sambanova import SambaStudio, Sambaverse
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import StructuredTool, ToolException, Tool, tool
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_experimental.utilities import PythonREPL
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.ai import AIMessage
from langchain_core.messages.tool import ToolMessage
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain.globals import set_debug

set_debug(False)

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, '..'))
repo_dir = os.path.abspath(os.path.join(kit_dir, '..'))

sys.path.append(kit_dir)
sys.path.append(repo_dir)

load_dotenv(os.path.join(repo_dir, '.env'))

True

## Tools Definitions

Here are defined several langchain tools and custom tools showing different. ways of implementing them

### Basic tools

The most simple way of implementing a custom langchain tool is using the `@tool` decorator and defining your arguments schema 

#### Get time tool

In [378]:
# tool schema
class GetTimeSchema(BaseModel):
    """Returns current date, current time or both."""

    kind: Optional[str] = Field(description='kind of information to retrieve "date", "time" or "both"')

In [379]:
# definition using @tool decorator
@tool(args_schema=GetTimeSchema)
def get_time(kind: str = 'both') -> str:
    """Returns current date, current time or both.

    Args:
        kind: date, time or both
    """
    if kind == 'date':
        date = datetime.now().strftime('%d/%m/%Y')
        return f'Current date: {date}'
    elif kind == 'time':
        time = datetime.now().strftime('%H:%M:%S')
        return f'Current time: {time}'
    else:
        date = datetime.now().strftime('%d/%m/%Y')
        time = datetime.now().strftime('%H:%M:%S')
        return f'Current date: {date}, Current time: {time}'

In [380]:
get_time.invoke({'kind': 'time'})

'Current time: 10:48:41'

In [381]:
get_time.get_input_schema().schema()

{'title': 'GetTimeSchema',
 'description': 'Returns current date, current time or both.',
 'type': 'object',
 'properties': {'kind': {'title': 'Kind',
   'description': 'kind of information to retrieve "date", "time" or "both"',
   'type': 'string'}}}

### Customized error handling tools

You can create a more complex tool that is able to raise specific tool error with some information, that can be send to the llm in order to fix the original call if its possible 

#### Calculator tool

In [382]:
# tool schema
class CalculatorSchema(BaseModel):
    """allow calculation of only basic operations: + - * and /
    with a string input expression"""

    expression: str = Field(..., description="expression to calculate, example '12 * 3'")

In [383]:
# function to use in the tool
def calculator(expression: str) -> Union[str, int, float]:
    """
    allow calculation of basic operations
    with a string input expression
    Args:
        expression: expression to calculate
    """
    ops = {
        '+': operator.add,
        '-': operator.sub,
        '*': operator.mul,
        'x': operator.mul,
        'X': operator.mul,
        '÷': operator.truediv,
        '/': operator.truediv,
    }
    tokens = re.findall(r'\d+\.?\d*|\+|\-|\*|\/|÷|x|X', expression)

    if len(tokens) == 0:
        raise ToolException(
            f"Invalid expression '{expression}', should only contain one of the following operators + - * x and ÷"
        )

    current_value = float(tokens.pop(0))

    while len(tokens) > 0:
        # The next token should be an operator
        op = tokens.pop(0)

        # The next token should be a number
        if len(tokens) == 0:
            raise ToolException(f"Incomplete expression '{expression}'")
        try:
            next_value = float(tokens.pop(0))

        except ValueError:
            raise ToolException('Invalid number format')

        except:
            raise ToolException('Invalid operation')

        # check division by 0
        if op in ['/', '÷'] and next_value == 0:
            raise ToolException('cannot divide by 0')

        current_value = ops[op](current_value, next_value)

    result = current_value

    return result


# tool error handler
def _handle_error(error: ToolException) -> str:
    return f'The following errors occurred during Calculator tool execution: `{error.args}`'


# tool definition
calculator = StructuredTool.from_function(
    func=calculator,
    args_schema=CalculatorSchema,
    handle_tool_error=_handle_error,  # set as True if you want the tool to trow a generic ToolError message "Tool execution error"
)

In [384]:
calculator.invoke('18*23.7 -5')

421.59999999999997

In [385]:
calculator.invoke('7 / 0')

"The following errors occurred during Calculator tool execution: `('cannot divide by 0',)`"

In [386]:
calculator.get_input_schema().schema()

{'title': 'CalculatorSchema',
 'description': 'allow calculation of only basic operations: + - * and /\nwith a string input expression',
 'type': 'object',
 'properties': {'expression': {'title': 'Expression',
   'description': "expression to calculate, example '12 * 3'",
   'type': 'string'}},
 'required': ['expression']}

### Langchain Tools

(There are several built-in tools available in langchain library that can be directly used by the model, you can find a list of available tools [here](https://python.langchain.com/v0.1/docs/integrations/tools/) 

#### Python standard shell, or REPL (Read-Eval-Print Loop) tool

In [387]:
# tool schema
class ReplSchema(BaseModel):
    "A Python shell. Use this to evaluate python commands. Input should be a valid python commands and expressions. If you want to see the output of a value, you should print it out with `print(...)`, if you need a specific module you should import it."

    command: str = Field(..., description='python code to evaluate')

In [388]:
# tool definition
python_repl = PythonREPL()
python_repl = Tool(
    name='python_repl',
    description='A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.',
    func=python_repl.run,
    args_schema=ReplSchema,
)

In [389]:
python_repl.invoke({'command': 'for i in range(0,5):\n\tprint(i)'})

'0\n1\n2\n3\n4\n'

In [390]:
python_repl.get_input_schema().schema()

{'title': 'ReplSchema',
 'description': 'A Python shell. Use this to evaluate python commands. Input should be a valid python commands and expressions. If you want to see the output of a value, you should print it out with `print(...)`, if you need a specific module you should import it.',
 'type': 'object',
 'properties': {'command': {'title': 'Command',
   'description': 'python code to evaluate',
   'type': 'string'}},
 'required': ['command']}

#### SQL calling tool

This tool is a demonstration of how to implement an SQL query tool leveraging the build in QuerySQLDataBaseTool langchain tool, for this example we are passing the db schema to Llama3 8B model, but you can get better performance with your own fine tuned model.
>For fine tuning your own sql model go to [fine_tuning_sql kit](../../fine_tuning_sql/README.md)

In [391]:
# example sql query call
db_path = os.path.join(kit_dir, 'data/chinook.db')
db_uri = f'sqlite:///{db_path}'
db = SQLDatabase.from_uri(db_uri)
print(db.get_usable_table_names())
print(db.run('SELECT * FROM genres;'))

['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'tracks']
[(1, 'Rock'), (2, 'Jazz'), (3, 'Metal'), (4, 'Alternative & Punk'), (5, 'Rock And Roll'), (6, 'Blues'), (7, 'Latin'), (8, 'Reggae'), (9, 'Pop'), (10, 'Soundtrack'), (11, 'Bossa Nova'), (12, 'Easy Listening'), (13, 'Heavy Metal'), (14, 'R&B/Soul'), (15, 'Electronica/Dance'), (16, 'World'), (17, 'Hip Hop/Rap'), (18, 'Science Fiction'), (19, 'TV Shows'), (20, 'Sci Fi & Fantasy'), (21, 'Drama'), (22, 'Comedy'), (23, 'Alternative'), (24, 'Classical'), (25, 'Opera')]


In [392]:
# tool schema
class QueryDBSchema(BaseModel):
    "A query generation tool. Use this to generate sql queries and retrieve the results from a database. Do not pass sql queries directly. Input must be a natural language question or instruction."

    query: str = Field(..., description='natural language question or instruction.')

In [430]:
def sql_finder(text):
    """Search in a string for a SQL query or code with format"""

    # regex for finding sql_code_pattern with format:
    # ```sql
    #    <query>
    # ```
    sql_code_pattern = re.compile(r'```sql\s+(.*?)\s+```', re.DOTALL)
    match = sql_code_pattern.search(text)
    if match is not None:
        query = match.group(1)
        return query
    else:
        # regex for finding sql_code_pattern with format:
        # ```
        # <quey>
        # ```
        code_pattern = re.compile(r'```\s+(.*?)\s+```', re.DOTALL)
        match = code_pattern.search(text)
        if match is not None:
            query = match.group(1)
            return query
        else:
            raise Exception('No SQL code found in LLM generation')


@tool(args_schema=QueryDBSchema)
def query_db(query):
    """A query generation tool. Use this to generate sql queries and retrieve the results from a database. Do not pass sql queries directly. Input must be a natural language question or instruction."""

    # Using Sambaverse expert as model for generating the SQL Query
    # llm = Sambaverse(
    #     sambaverse_model_name='Meta/Meta-Llama-3-8B-Instruct',
    #     streaming=True,
    #     model_kwargs={
    #         'max_tokens_to_generate': 512,
    #         'select_expert': 'Meta-Llama-3-8B-Instruct',
    #         'temperature': 0.0,
    #         'repetition_penalty': 1.0,
    #         'top_k': 1,
    #         'top_p': 1.0,
    #         'do_sample': False,
    #         'process_prompt': True,
    #     },
    # )

    # Using SambaStudio CoE expert as model for generating the SQL Query
    llm = SambaStudio(
        streaming=True,
        model_kwargs={
            'max_tokens_to_generate': 512,
            'select_expert': 'Meta-Llama-3-8B-Instruct',
            'temperature': 0.0,
            'repetition_penalty': 1.0,
            'top_k': 1,
            'top_p': 1.0,
            'do_sample': False,
        },
    )

    prompt = PromptTemplate.from_template(
        """<|begin_of_text|><|start_header_id|>system<|end_header_id|> 
        
        {table_info}
        
        Generate a query using valid SQLite to answer the following questions for the summarized tables schemas provided above.
        Do not assume the values on the database tables before generating the SQL query, always generate a SQL that query what is asked.
        The query must be in the format: ```sql\nquery\n```
        
        Example:
        
        ```sql
        SELECT * FROM mainTable;
        ```
        
        <|eot_id|><|start_header_id|>user<|end_header_id|>\
            
        {input}
        <|eot_id|><|start_header_id|>assistant<|end_header_id|>"""
    )

    # Chain that receives the natural language input and the table schema, then pass the teh formmated prompt to the llm
    # and finally execute the sql finder method, retrieving only the filtered SQL query
    query_generation_chain = prompt | llm | RunnableLambda(sql_finder)
    table_info = db.get_table_info()
    query = query_generation_chain.invoke({'input': query, 'table_info': table_info})
    print(query)
    query_executor = QuerySQLDataBaseTool(db=db)
    result = query_executor.invoke(query)

    result = f'Query {query} executed with result {result}'

    return result

##### Examples of query_db tool call

In [431]:
query_db.invoke({'query': 'How many genres of music are in the chinook db'})

SELECT COUNT(*) 
FROM genres;


'Query SELECT COUNT(*) \nFROM genres; executed with result [(25,)]'

In [313]:
query_db.invoke({'query': 'add a new genre in the chinook db called Salsa'})

INSERT INTO genres (GenreId, Name)
VALUES ((SELECT MAX(GenreId) + 1 FROM genres), 'Salsa');


"Query INSERT INTO genres (GenreId, Name)\nVALUES ((SELECT MAX(GenreId) + 1 FROM genres), 'Salsa'); executed with result "

In [314]:
query_db.invoke({'query': 'How many genres of music are in the chinook db'})

SELECT COUNT(*) 
FROM genres;


'Query SELECT COUNT(*) \nFROM genres; executed with result [(26,)]'

In [315]:
query_db.invoke({'query': 'What is the longest track in the chinook db'})

SELECT t.Name, t.Milliseconds
FROM tracks t
ORDER BY t.Milliseconds DESC
LIMIT 1;


"Query SELECT t.Name, t.Milliseconds\nFROM tracks t\nORDER BY t.Milliseconds DESC\nLIMIT 1; executed with result [('Occupation / Precipice', 5286953)]"

In [316]:
query_db.invoke({'query': 'list of the 5 highest value invoices registered in the database'})

SELECT * 
FROM invoices 
ORDER BY Total DESC 
LIMIT 5;


"Query SELECT * \nFROM invoices \nORDER BY Total DESC \nLIMIT 5; executed with result [(404, 6, '2013-11-13 00:00:00', 'Rilská 3174/6', 'Prague', None, 'Czech Republic', '14300', 25.86), (299, 26, '2012-08-05 00:00:00', '2211 W Berry Street', 'Fort Worth', 'TX', 'USA', '76110', 23.86), (96, 45, '2010-02-18 00:00:00', 'Erzsébet krt. 58.', 'Budapest', None, 'Hungary', 'H-1073', 21.86), (194, 46, '2011-04-28 00:00:00', '3 Chatham Street', 'Dublin', 'Dublin', 'Ireland', None, 21.86), (89, 7, '2010-01-18 00:00:00', 'Rotenturmstraße 4, 1010 Innere Stadt', 'Vienne', None, 'Austria', '1010', 18.86)]"

In [336]:
query_db.invoke(
    {
        'query': 'add in the db a new song called "El Preso" from the artist "Fruco y sus tesos", with a duration 123000 seconds with a price of 1.2 usd'
    }
)

INSERT INTO tracks (Name, AlbumId, MediaTypeId, GenreId, Composer, Milliseconds, Bytes, UnitPrice)
VALUES ('El Preso', NULL, 1, 3, 'Fruco y sus tesos', 123000, NULL, 1.2);


"Query INSERT INTO tracks (Name, AlbumId, MediaTypeId, GenreId, Composer, Milliseconds, Bytes, UnitPrice)\nVALUES ('El Preso', NULL, 1, 3, 'Fruco y sus tesos', 123000, NULL, 1.2); executed with result "

In [337]:
query_db.invoke({'query': 'who is the composer of the song with name "El Preso"'})

SELECT Composer
FROM tracks
WHERE Name = 'El Preso';


"Query SELECT Composer\nFROM tracks\nWHERE Name = 'El Preso'; executed with result [('Fruco y sus tesos',)]"

In [321]:
query_db.invoke(
    {'query': 'create a new table in the chinook db called movies, with a column for id and column for movieTitle'}
)

CREATE TABLE movies (
    "MovieId" INTEGER PRIMARY KEY,
    "MovieTitle" NVARCHAR(200) NOT NULL
);


'Query CREATE TABLE movies (\n    "MovieId" INTEGER PRIMARY KEY,\n    "MovieTitle" NVARCHAR(200) NOT NULL\n); executed with result '

In [325]:
# for exectuing this query it is needed to reinitialize the query db to get the new schema including movies table
query_db.invoke({'query': 'add a new register in movies table for "star wars"'})

INSERT INTO movies (MovieId, MovieTitle)
VALUES (NULL, 'Star Wars');


"Query INSERT INTO movies (MovieId, MovieTitle)\nVALUES (NULL, 'Star Wars'); executed with result "

### Default response tool

We define a default tool to return conversational responses to the user, for this we will only define the schema of the tool and manually handle the behavior of the system when this tool is called by the model

In [463]:
# tool schema
class ConversationalResponse(BaseModel):
    "Respond conversationally only if no other tools should be called for a given query, or if you have a final answer. response must be in the same language as the user query"

    response: str = Field(
        ..., description='Conversational response to the user. must be in the same language as the user query'
    )


ConversationalResponse.schema()

{'title': 'ConversationalResponse',
 'description': 'Respond conversationally only if no other tools should be called for a given query, or if you have a final answer. response must be in the same language as the user query',
 'type': 'object',
 'properties': {'response': {'title': 'Response',
   'description': 'Conversational response to the user. must be in the same language as the user query',
   'type': 'string'}},
 'required': ['response']}

In [464]:
def get_tools_schemas(tools: Union[Tool, list] = None, default: Union[Tool, BaseModel] = None):
    if tools is None or isinstance(tools, list):
        pass
    elif isinstance(tools, Tool) or isinstance(tools, StructuredTool):
        tools = [tools]
    else:
        raise TypeError('tools must be a Tool or a list of Tools')

    tools_schemas = []

    for tool in tools:
        tool_schema = tool.get_input_schema().schema()
        schema = {'name': tool.name, 'description': tool_schema['description'], 'properties': tool_schema['properties']}
        if 'required' in schema:
            schema['required'] = tool_schema['required']
        tools_schemas.append(schema)

    if default is not None:
        if isinstance(default, Tool) or isinstance(default, StructuredTool):
            tool_schema = default.get_input_schema().schema()
        elif issubclass(default, BaseModel):
            tool_schema = default.schema()
        else:
            raise TypeError('default must be a Tool or a BaseModel')
        schema = {
            'name': tool_schema['title'],
            'description': tool_schema['description'],
            'properties': tool_schema['properties'],
        }
        if 'required' in schema:
            schema['required'] = tool_schema['required']
        tools_schemas.append(schema)

    return tools_schemas

#### Set of tools

In [465]:
tools = [get_time, calculator, python_repl, query_db]

In [466]:
tools_schemas = get_tools_schemas(tools, default=ConversationalResponse)
tools_schemas = '\n'.join([json.dumps(tool, indent=2) for tool in tools_schemas])
pprint(tools_schemas)

('{\n'
 '  "name": "get_time",\n'
 '  "description": "Returns current date, current time or both.",\n'
 '  "properties": {\n'
 '    "kind": {\n'
 '      "title": "Kind",\n'
 '      "description": "kind of information to retrieve \\"date\\", \\"time\\" '
 'or \\"both\\"",\n'
 '      "type": "string"\n'
 '    }\n'
 '  }\n'
 '}\n'
 '{\n'
 '  "name": "calculator",\n'
 '  "description": "allow calculation of only basic operations: + - * and '
 '/\\nwith a string input expression",\n'
 '  "properties": {\n'
 '    "expression": {\n'
 '      "title": "Expression",\n'
 '      "description": "expression to calculate, example \'12 * 3\'",\n'
 '      "type": "string"\n'
 '    }\n'
 '  }\n'
 '}\n'
 '{\n'
 '  "name": "python_repl",\n'
 '  "description": "A Python shell. Use this to evaluate python commands. '
 'Input should be a valid python commands and expressions. If you want to see '
 'the output of a value, you should print it out with `print(...)`, if you '
 'need a specific module you should 

## Function Calling 

### LLM definition

In [467]:
# Using Sambaverse CoE expert as model for tool calling
# llm = Sambaverse(
#     sambaverse_model_name='Meta/Meta-Llama-3-70B-Instruct',
#     model_kwargs={
#         'max_tokens_to_generate': 2048,
#         'select_expert': 'Meta-Llama-3-70B-Instruct',
#         'process_prompt': True,
#         'temperature': 0.01,
#     },
# )

# Using SambaStudio CoE expert as model for tool calling
llm = SambaStudio(
    streaming=True,
    model_kwargs={
        'max_tokens_to_generate': 2048,
        'select_expert': 'Meta-Llama-3-70B-Instruct',
        'process_prompt': False,
    },
)

### Tool execution

Definition of som util methods to parse the llm output and invoke available tools

In [468]:
tools_map = {'get_time': get_time, 'calculator': calculator, 'python_repl': python_repl, 'query_db': query_db}


def execute(tools):
    """
    Given a list of tool executions the llm return as required
    execute them given the name with the mane in tools_map and the input arguments
    if there is only one tool call and it is default conversational one, the response is marked as final response
    """
    tool_msg = "Tool '{name}'response: {response}"
    tools_msgs = []
    if len(tools) == 1 and tools[0]['tool'].lower() == 'conversationalresponse':
        final_answer = True
        return final_answer, tools[0]['tool_input']['response']
    for tool in tools:
        final_answer = False
        if tool['tool'].lower() != 'conversationalresponse':
            response = tools_map[tool['tool'].lower()].invoke(tool['tool_input'])
            tools_msgs.append(tool_msg.format(name=tool['tool'], response=str(response)))
    return final_answer, tools_msgs

In [469]:
def jsonFinder(input_string):
    """
    find json structures ina  llm string response, if bad formatted using LLM to correct it
    """
    json_pattern = re.compile(r'(\{.*\}|\[.*\])', re.DOTALL)
    # Find the first JSON structure in the string
    json_match = json_pattern.search(input_string)
    if json_match:
        json_str = json_match.group(1)
        try:
            json.loads(json_str)
        except:
            json_correction_prompt = """|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a json format corrector tool<|eot_id|><|start_header_id|>user<|end_header_id|>
            fix the following json file: {json} 
            <|eot_id|><|start_header_id|>assistant<|end_header_id|>
            fixed json: """
            json_correction_prompt_template = PromptTemplate.from_template(json_correction_prompt)
            json_correction_chain = json_correction_prompt_template | llm
            json_str = json_correction_chain.invoke(json_str)
    else:
        # implement here not finding json format parsing to json or error rising
        json_str = None
    return json_str

Agentic prompt template for function calling

In [470]:
example_function_calling_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an helpful assistant and you have access to the following tools:

{tools}

You must always select one or more of the above tools and answer with only a list of JSON objects matching the following schema:

```json
[{{
  "tool": <name of the selected tool>,
  "tool_input": <parameters for the selected tool, matching the tool's JSON schema>
}}]
```

Think step by step
Do not call a tool if the input depends on another tool output you dont have yet.
Do not try to answer until you get tools output, if you dont have an answer yet you can continue calling tools until you do..
Your answer should be in the same language as the initial query.

<|eot_id|><|start_header_id|>user<|end_header_id|>
User: {usr_msg} 
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Assistant:"""

function_calling_prompt_template = PromptTemplate.from_template(example_function_calling_prompt)

Json parsing Chain for parsing tool calling output from llm

In [471]:
json_parsing_chain = RunnableLambda(jsonFinder) | JsonOutputParser()

Chain for passing through the tools schemas to the model

In [472]:
prompt_template = {
    'tools': lambda x: tools_schemas,
    'usr_msg': RunnablePassthrough(),
} | function_calling_prompt_template

Example of chin to do one full pass of the query to the model 
Here we will see the first tool execution call generated by the model and the result of this first execution 

In [473]:
default_fc_chain = prompt_template | llm | json_parsing_chain

In [445]:
query = 'hi'
response_tools = default_fc_chain.invoke(query)
print(response_tools)
final_answer, response_tools = execute(response_tools)
print(response_tools)

[{'tool': 'ConversationalResponse', 'tool_input': {'response': 'Hi! How can I assist you today?'}}]
Hi! How can I assist you today?


In [446]:
query = 'it is time to go to sleep?'
response_tools = default_fc_chain.invoke(query)
pprint(response_tools)
final_answer, response_tools = execute(response_tools)
print(response_tools)

[{'tool': 'get_time', 'tool_input': {'kind': 'time'}}]
["Tool 'get_time'response: Current time: 10:58:40"]


In [354]:
query = 'whats is 347 min in hours and minutes?'
response_tools = default_fc_chain.invoke(query)
print(response_tools)
final_answer, response_tools = execute(response_tools)
print(response_tools)

[{'tool': 'calculator', 'tool_input': {'expression': '347 / 60'}}]
["Tool 'calculator'response: 5.783333333333333"]


In [355]:
query = "is this word is a palindrome? 'saippuakivikauppias'"
response_tools = default_fc_chain.invoke(query)
pprint(response_tools)
final_answer, response_tools = execute(response_tools)
print(response_tools)

[{'tool': 'python_repl',
  'tool_input': {'command': "print('saippuakivikauppias' "
                            "=='saippuakivikauppias'[::-1])"}}]
["Tool 'python_repl'response: True\n"]


In [356]:
query = "sort this list of elements alphabetically ['screwdriver', 'pliers', 'hammer']"
response_tools = default_fc_chain.invoke(query)
pprint(response_tools)
final_answer, response_tools = execute(response_tools)
print(response_tools)

[{'tool': 'python_repl',
  'tool_input': {'command': "print(sorted(['screwdriver', 'pliers', "
                            "'hammer']))"}}]
["Tool 'python_repl'response: ['hammer', 'pliers', 'screwdriver']\n"]


### Function Calling pipeline 

Here we are defining a iterative pipeline to give the model the ability of getting the tools execution as inputs and continue generating the answer until having a final response 

Definition of the system message

In [480]:
function_calling_system_prompt = """you are an helpful assistant and you have access to the following tools:

{tools}

You must always select one or more of the above tools and answer with only a list of JSON objects matching the following schema:

```json
[{{
  "tool": <name of the selected tool>,
  "tool_input": <parameters for the selected tool, matching the tool's JSON schema>
}}]
```

Think step by step
Do not call a tool if the input depends on another tool output that you do not have yet
Do not try to answer until you get all the tools output, if you do not have an answer yet, you can continue calling tools until you do.
Your answer should be in the same language as the initial query.

"""

Creation of chat prompt template, having as first interaction the system prompt

In [481]:
function_calling_chat_template = ChatPromptTemplate.from_messages([('system', function_calling_system_prompt)])
function_calling_chat_template

ChatPromptTemplate(input_variables=['tools'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tools'], template='you are an helpful assistant and you have access to the following tools:\n\n{tools}\n\nYou must always select one or more of the above tools and answer with only a list of JSON objects matching the following schema:\n\n```json\n[{{\n  "tool": <name of the selected tool>,\n  "tool_input": <parameters for the selected tool, matching the tool\'s JSON schema>\n}}]\n```\n\nThink step by step\nDo not call a tool if the input depends on another tool output that you do not have yet\nDo not try to answer until you get all the tools output, if you do not have an answer yet, you can continue calling tools until you do.\nYour answer should be in the same language as the initial query.\n\n'))])

Utility to add convert and format each interaction from the user, model or tool to a langchain message with role

In [482]:
def msgs_to_llama3_str(msgs: list):
    """
    convert a list of langchain messages with roles to expected LLmana 3 input
    """
    formatted_msgs = []
    for msg in msgs:
        if msg.type == 'system':
            sys_placeholder = '<|begin_of_text|><|start_header_id|>system<|end_header_id|> {msg}'
            formatted_msgs.append(sys_placeholder.format(msg=msg.content))
        elif msg.type == 'human':
            human_placeholder = '<|eot_id|><|start_header_id|>user<|end_header_id|>\nUser: {msg} <|eot_id|><|start_header_id|>assistant<|end_header_id|>\nAssistant:'
            formatted_msgs.append(human_placeholder.format(msg=msg.content))
        elif msg.type == 'ai':
            assistant_placeholder = '<|eot_id|><|start_header_id|>assistant<|end_header_id|>\nAssistant: {msg}'
            formatted_msgs.append(assistant_placeholder.format(msg=msg.content))
        elif msg.type == 'tool':
            tool_placeholder = '<|eot_id|><|start_header_id|>tools<|end_header_id|>\n{msg} <|eot_id|><|start_header_id|>assistant<|end_header_id|>\nAssistant:'
            formatted_msgs.append(tool_placeholder.format(msg=msg.content))
        else:
            raise ValueError(f'Invalid message type: {msg.type}')
    return '\n'.join(formatted_msgs)

Putting everything together to have an iterative/agentic pipeline for doing function calling

In [483]:
def function_call_llm(query, max_it=5, debug=False):
    """
    invocation method for function calling workflow
    """
    history = function_calling_chat_template.format_prompt(tools=tools_schemas).to_messages()
    history.append(HumanMessage(query))
    tool_call_id = 0  # identification for each tool calling required to create ToolMessages

    for i in range(max_it):
        prompt = msgs_to_llama3_str(history)
        llm_response = llm.invoke(prompt)
        parsed_tools_llm_response = json_parsing_chain.invoke(llm_response)
        history.append(AIMessage(llm_response))
        final_answer, tools_msgs = execute(parsed_tools_llm_response)
        if final_answer:
            final_response = tools_msgs
            if debug:
                pprint(history)
            return final_response
        else:
            history.append(ToolMessage('\n'.join(tools_msgs), tool_call_id=tool_call_id))
            tool_call_id += 1

    raise Exception('not a final response yet', json.dumps(history))

In [456]:
response = function_call_llm('what time is it?', max_it=5, debug=True)

[SystemMessage(content='you are an helpful assistant and you have access to the following tools:\n\n{\n  "name": "get_time",\n  "description": "Returns current date, current time or both.",\n  "properties": {\n    "kind": {\n      "title": "Kind",\n      "description": "kind of information to retrieve \\"date\\", \\"time\\" or \\"both\\"",\n      "type": "string"\n    }\n  }\n}\n{\n  "name": "calculator",\n  "description": "allow calculation of only basic operations: + - * and /\\nwith a string input expression",\n  "properties": {\n    "expression": {\n      "title": "Expression",\n      "description": "expression to calculate, example \'12 * 3\'",\n      "type": "string"\n    }\n  }\n}\n{\n  "name": "python_repl",\n  "description": "A Python shell. Use this to evaluate python commands. Input should be a valid python commands and expressions. If you want to see the output of a value, you should print it out with `print(...)`, if you need a specific module you should import it.",\n  "p

In [457]:
print(response)

The current date is 25/06/2024 and the time is 11:01:30.


In [458]:
response = function_call_llm('it is time to go to sleep, how many hours last to 10pm?', max_it=5, debug=True)

[SystemMessage(content='you are an helpful assistant and you have access to the following tools:\n\n{\n  "name": "get_time",\n  "description": "Returns current date, current time or both.",\n  "properties": {\n    "kind": {\n      "title": "Kind",\n      "description": "kind of information to retrieve \\"date\\", \\"time\\" or \\"both\\"",\n      "type": "string"\n    }\n  }\n}\n{\n  "name": "calculator",\n  "description": "allow calculation of only basic operations: + - * and /\\nwith a string input expression",\n  "properties": {\n    "expression": {\n      "title": "Expression",\n      "description": "expression to calculate, example \'12 * 3\'",\n      "type": "string"\n    }\n  }\n}\n{\n  "name": "python_repl",\n  "description": "A Python shell. Use this to evaluate python commands. Input should be a valid python commands and expressions. If you want to see the output of a value, you should print it out with `print(...)`, if you need a specific module you should import it.",\n  "p

In [459]:
response

'You have 10 hours and 59 minutes left until 10pm.'

In [365]:
response = function_call_llm("is this word is a palindrome? 'saippuakivikauppias'", max_it=5, debug=True)

[SystemMessage(content='you are an helpful assistant and you have access to the following tools:\n\n{\n  "name": "get_time",\n  "description": "Returns current date, current time or both.",\n  "properties": {\n    "kind": {\n      "title": "Kind",\n      "description": "kind of information to retrieve \\"date\\", \\"time\\" or \\"both\\"",\n      "type": "string"\n    }\n  }\n}\n{\n  "name": "calculator",\n  "description": "allow calculation of only basic operations: + - * and /\\nwith a string input expression",\n  "properties": {\n    "expression": {\n      "title": "Expression",\n      "description": "expression to calculate, example \'12 * 3\'",\n      "type": "string"\n    }\n  }\n}\n{\n  "name": "python_repl",\n  "description": "A Python shell. Use this to evaluate python commands. Input should be a valid python commands and expressions. If you want to see the output of a value, you should print it out with `print(...)`, if you need a specific module you should import it.",\n  "p

In [366]:
response

"Yes, the word'saippuakivikauppias' is a palindrome."

In [367]:
response = function_call_llm(
    "sort this list of elements alphabetically ['screwdriver', 'pliers', 'hammer']", max_it=5, debug=True
)

[SystemMessage(content='you are an helpful assistant and you have access to the following tools:\n\n{\n  "name": "get_time",\n  "description": "Returns current date, current time or both.",\n  "properties": {\n    "kind": {\n      "title": "Kind",\n      "description": "kind of information to retrieve \\"date\\", \\"time\\" or \\"both\\"",\n      "type": "string"\n    }\n  }\n}\n{\n  "name": "calculator",\n  "description": "allow calculation of only basic operations: + - * and /\\nwith a string input expression",\n  "properties": {\n    "expression": {\n      "title": "Expression",\n      "description": "expression to calculate, example \'12 * 3\'",\n      "type": "string"\n    }\n  }\n}\n{\n  "name": "python_repl",\n  "description": "A Python shell. Use this to evaluate python commands. Input should be a valid python commands and expressions. If you want to see the output of a value, you should print it out with `print(...)`, if you need a specific module you should import it.",\n  "p

In [368]:
response

"The sorted list is: ['hammer', 'pliers','screwdriver']"

In [375]:
response = function_call_llm(
    'whats the price in colombian pesos of the track "Snowballed" in the db if one usd is equal to 3800 cop?',
    max_it=5,
    debug=True,
)

SELECT UnitPrice * 0.76 AS PriceUSD
FROM tracks
WHERE Name = 'Snowballed';
[SystemMessage(content='you are an helpful assistant and you have access to the following tools:\n\n{\n  "name": "get_time",\n  "description": "Returns current date, current time or both.",\n  "properties": {\n    "kind": {\n      "title": "Kind",\n      "description": "kind of information to retrieve \\"date\\", \\"time\\" or \\"both\\"",\n      "type": "string"\n    }\n  }\n}\n{\n  "name": "calculator",\n  "description": "allow calculation of only basic operations: + - * and /\\nwith a string input expression",\n  "properties": {\n    "expression": {\n      "title": "Expression",\n      "description": "expression to calculate, example \'12 * 3\'",\n      "type": "string"\n    }\n  }\n}\n{\n  "name": "python_repl",\n  "description": "A Python shell. Use this to evaluate python commands. Input should be a valid python commands and expressions. If you want to see the output of a value, you should print it out wit